# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
#jupyter nbextension enable py gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests

import os

# Import API key
from api_keys import g_key

In [2]:
import gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
data = "../output_data/WeatherPy_Log.csv"
data_df= pd.read_csv(data)
data_df

,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed
0,La Ronge,CA,6050066,55.10,-105.28,39.20,27,75,10.29
1,Antón Lizardo,MX,3532821,19.05,-95.98,86.00,66,40,13.87
2,Albany,US,5106841,42.60,-73.97,46.99,97,99,12.84
3,Borogontsy,RU,2026160,62.67,131.17,32.79,94,100,5.73
4,Bluff,NZ,2206939,-46.60,168.33,49.91,76,46,11.50
...,...,...,...,...,...,...,...,...,...
562,Andrychów,PL,3104115,49.85,19.34,51.80,93,75,20.80
563,Keti Bandar,PK,1174451,24.14,67.45,80.67,88,90,12.86
564,Abu Dhabi,AE,292968,24.47,54.37,86.00,51,23,12.75
565,Ibrā’,OM,287832,22.69,58.53,81.95,54,2,2.37


In [10]:
data = pd.DataFrame.to_html(data_df)
data

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>City Name</th>\n      <th>Country</th>\n      <th>City ID</th>\n      <th>Latitude</th>\n      <th>Longitude</th>\n      <th>Max Temperature</th>\n      <th>Humidity</th>\n      <th>Cloudiness (%)</th>\n      <th>Wind Speed</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>La Ronge</td>\n      <td>CA</td>\n      <td>6050066</td>\n      <td>55.10</td>\n      <td>-105.28</td>\n      <td>39.20</td>\n      <td>27</td>\n      <td>75</td>\n      <td>10.29</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Antón Lizardo</td>\n      <td>MX</td>\n      <td>3532821</td>\n      <td>19.05</td>\n      <td>-95.98</td>\n      <td>86.00</td>\n      <td>66</td>\n      <td>40</td>\n      <td>13.87</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Albany</td>\n      <td>US</td>\n      <td>5106841</td>\n      <td>42.60</td>\n      <td>-73.97</td>\n      <td

In [12]:
with open('Data.html', 'w') as fo:
    data_df.to_html(fo)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
gmaps.configure(api_key=g_key)
data_df = data_df.dropna()

locations = data_df[["Latitude", "Longitude"]].astype(float)
humidity = data_df["Humidity"].astype(float)
type(locations)

pandas.core.frame.DataFrame

In [24]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
Hotel_df = data_df.loc[(data_df["Humidity"]<70)& (data_df["Max Temperature"]<55)& (data_df["Max Temperature"]
                                       >25) & (data_df["Wind Speed"]<5) & (data_df["Cloudiness (%)"]<10)]

Hotel_df

,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed
104,Linxia Chengguanzhen,CN,1803331,35.60,103.21,44.94,57,0,2.04
250,Karakol,KG,1528121,42.49,78.39,49.48,50,0,3.27
372,Kamennomostskiy,RU,553514,44.30,40.19,47.35,68,0,4.90
445,Mount Isa,AU,2065594,-20.73,139.50,44.60,45,0,1.12
530,Marquette,US,5000950,46.53,-87.63,39.99,39,0,4.00
536,Mehrān,IR,124291,33.12,46.16,51.80,50,0,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
# FIND THE HOTEL INFORMATION. THIS IS THE SHORTENED CODE, I COMMENTED OUT A BIT
# lat_list = []
# long_list = []
# coordinates_list = []
#Hotel_Location_List = []

Hotel_Name_List = []

locations = Hotel_df[["Latitude", "Longitude"]].astype(float)
for x in range (len(locations)):
    lat=locations.iloc[x,0]
    long=locations.iloc[x,1]
#     lat_list.append(lat)
#     long_list.append(long)
#     coordinates = str(lat_list[x]) + "," + str(long_list[x]) 
#     coordinates_list.append(coordinates)
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params = {
            "location": f"{lat},{long}",
            "radius": 5000,
            "keyword" : "hotel",
            "type": "hotel",
            "key": g_key,
    }


    response = requests.get(base_url, params=params).json()
    results=response["results"]
    try: 
        Hotel_Name = results[1]["name"]  
        Hotel_Name_List.append(Hotel_Name)
#         Hotel_Location = results[0]['plus_code']["compound_code"]
#         Hotel_Location_List.append(Hotel_Location)
    except:
        print(f"{lat},{long}, No results for this location")
        
        No_Hotel = "No Hotel near this location"
        Hotel_Name_List.append(No_Hotel)

print(Hotel_Name_List)


33.12,46.16, No results for this location
['Linxia Hotel', 'Snow Leopard Hostel', 'Отдых в горах Адыгеи - Барская усадьба', 'Burke & Wills Motel Mt Isa', 'Jasper Ridge Inn Ishpeming', 'No Hotel near this location']


In [27]:
Hotel_df["Hotel Name"] = Hotel_Name_List
#Hotel = Hotel_Name_List
# Hotel_df.loc[Hotel_df["Hotel Name"] == Hotel_Name_List]
# Hotel_df
Hotel_df


/Users/imacmattimacmatt/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed,Hotel Name
104,Linxia Chengguanzhen,CN,1803331,35.60,103.21,44.94,57,0,2.04,Linxia Hotel
250,Karakol,KG,1528121,42.49,78.39,49.48,50,0,3.27,Snow Leopard Hostel
372,Kamennomostskiy,RU,553514,44.30,40.19,47.35,68,0,4.90,Отдых в горах Адыгеи - Барская усадьба
445,Mount Isa,AU,2065594,-20.73,139.50,44.60,45,0,1.12,Burke & Wills Motel Mt Isa
530,Marquette,US,5000950,46.53,-87.63,39.99,39,0,4.00,Jasper Ridge Inn Ishpeming
536,Mehrān,IR,124291,33.12,46.16,51.80,50,0,2.24,No Hotel near this location


In [28]:
# # # # #MAPPING

Hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=4,
    info_box_content= [f"Hotel Name: {name} " for name in Hotel_Name_List]
)


fig = gmaps.figure()
fig.add_layer(Hotel_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [29]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(Hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

In [101]:
# THIS IS THE PART I TALKED TO MARK ABOUT AND THEN SKIPPED

# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City Name}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
# locations = Hotel_df[["Latitude", "Longitude"]]